The goal is to write a simple object detection pipeline invoking YOLOX.

In [2]:
import numpy as np
import torch
import torchvision
import cv2

There are many YOLOX versions based on size, depth and efficiency and we select which yolox version to use by using the `get_exp` function provided by YOLOX.

In [6]:
import sys
sys.path.insert(0, "./YOLOX/")
from YOLOX.yolox.exp.build import get_exp_by_name

In [10]:
YOLOX_VERSION = "yolox-s" #alternatives: yolox-m, yolox-l, yolox-x, yolox-tiny, yolox-nano, yolov3
yolo_exp = get_exp_by_name(YOLOX_VERSION)

In [21]:
a = torch.from_numpy(np.array([[[1,2],[3,4]],[[5,6], [7,8]]]))


In [22]:
x,y = torch.max(a, dim=2, keepdim=False)

x,y

(tensor([[2, 4],
         [6, 8]]),
 tensor([[1, 1],
         [1, 1]]))

In [23]:
mask = x > 2

In [41]:

filter_classes = torch.tensor([1,5,11])

b = torch.arange(12).reshape((6,2))
c = b[:,1]

In [42]:
mask = torch.isin(c, filter_classes)

In [55]:
mask &= torch.tensor([True, False, False, False, True, True])

In [56]:
mask

tensor([ True, False, False, False, False,  True])

In [57]:
torch.stack([torch.tensor([1,2,3]), torch.tensor([4,5,6])], dim=1)

tensor([[1, 4],
        [2, 5],
        [3, 6]])

In [44]:
b[mask]

tensor([[ 0,  1],
        [ 4,  5],
        [10, 11]])

In [49]:
a = torch.tensor([1,2,3]).to(torch.device("cpu"),dtype=torch.float32)

In [54]:
a[:,None]

tensor([[1.],
        [2.],
        [3.]])